In [5]:
np.random.normal(0, 1,1)

array([-1.93879991])

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from causalml.inference.tree import UpliftTreeClassifier
from causalml.inference.meta import BaseTLearner
from sklearn.metrics import classification_report, roc_auc_score
import lightgbm as lgb

# Синтетический набор данных (можно заменить на реальный)
def generate_synthetic_data(n=5000):
    np.random.seed(42)
    X1 = np.random.normal(0, 1, n)
    X2 = np.random.normal(0, 1, n)
    X3 = np.random.normal(0, 1, n)

    # Фичи
    X = pd.DataFrame({"X1": X1, "X2": X2, "X3": X3})

    # Третий признак влияет на вероятность записи лечения
    treatment_effect = (X3 > 0).astype(int)
    treatment = np.random.binomial(1, 0.5 + 0.2 * treatment_effect)

    # Целевая метка
    y = np.random.binomial(1, 0.3 + 0.4 * treatment * np.clip(X1 + X2, 0, 1))

    return X, treatment, y

# Генерация данных
X, treatment, y = generate_synthetic_data()

# Разделение тренировочного и тестового наборов данных
X_train, X_test, treatment_train, treatment_test, y_train, y_test = train_test_split(
    X, treatment, y, test_size=0.3, random_state=42
)

# Используем T-Learner
t_learner = BaseTLearner(learner=RandomForestClassifier(random_state=42))

# Обучение T-Learner
t_learner.fit(X=X_train.values, treatment=treatment_train, y=y_train)

tmodel = tlearner.BaseTClassifier(
    # learner=lgb.LGBMClassifier(max_depth=4),
    control_learner=lgb.LGBMClassifier(),
    treatment_learner=lgb.LGBMClassifier(),
    control_name=0
)

# # Прогноз uplift (оставим для примера)
# uplift_pred = t_learner.predict(X_test.values)

# # Оценка качества модели - бинарная классификация по группам
# # Модель для контрольной группы (treatment=0)
# model_c = RandomForestClassifier()
# model_c.fit(X_train[treatment_train == 0], y_train[treatment_train == 0])

# # Модель для тестовой группы (treatment=1)
# model_t = RandomForestClassifier()
# model_t.fit(X_train[treatment_train == 1], y_train[treatment_train == 1])

# # Предсказания
# y_pred_control = model_c.predict(X_test)
# y_pred_treatment = model_t.predict(X_test)

# # Оценка - например, через AUC для каждой группы
# auc_control = roc_auc_score(y_test[treatment_test == 0], y_pred_control[treatment_test == 0])
# auc_treatment = roc_auc_score(y_test[treatment_test == 1], y_pred_treatment[treatment_test == 1])

# print(f"AUC для контрольной группы: {auc_control:.2f}")
# print(f"AUC для тестовой группы (treatment): {auc_treatment:.2f}")

# print("Пример uplift:")
# print(uplift_pred[:10])

AUC для контрольной группы: 0.49
AUC для тестовой группы (treatment): 0.63
Пример uplift:
[[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [9]:
import lightgbm as lgb
import causalml.inference.meta.tlearner as tlearner

In [ ]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))

# %load_ext autoreload
# %autoreload 2
import pandas as pd
from src.distributions_check import check_feature_distributions_by_stat_test, check_feature_distributions_by_model
from src.datasets import sample_features, TorchDataset, NumpyDataset

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from causalml.inference.tree import UpliftTreeClassifier
from causalml.inference.meta import BaseTLearner
from sklearn.metrics import classification_report, roc_auc_score
import causalml.inference.meta.tlearner as tlearner
import lightgbm as lgb


def generate_synthetic_data(n=5000):
    np.random.seed(42)
    X1 = np.random.normal(0, 1, n)
    X2 = np.random.normal(0, 1, n)
    X3 = np.random.normal(0, 1, n)

    # Фичи
    X = pd.DataFrame({"X1": X1, "X2": X2, "X3": X3})

    # Третий признак влияет на вероятность записи лечения
    treatment_effect = (X3 > 0).astype(int)
    treatment = np.random.binomial(1, 0.5 + 0.2 * treatment_effect)

    # Целевая метка
    y = np.random.binomial(1, 0.3 + 0.4 * treatment * np.clip(X1 + X2, 0, 1))

    return X, treatment, y

# Генерация данных
X, treatment, y = generate_synthetic_data()
tmodel = tlearner.BaseTClassifier(
    # learner=lgb.LGBMClassifier(max_depth=4),
    control_learner=lgb.LGBMClassifier(max_depth=2),
    treatment_learner=lgb.LGBMClassifier(max_depth=2),
    control_name=0
)
tmodel.fit(X=X, treatment=treatment, y=y)

In [10]:
tmodel = tlearner.BaseTClassifier(
    # learner=lgb.LGBMClassifier(max_depth=4),
    control_learner=lgb.LGBMClassifier(),
    treatment_learner=lgb.LGBMClassifier(),
    control_name=0
)


In [18]:
treatment_train.shape, y_train.shape, X_train.values.shape

((3500,), (3500,), (3500, 3))

In [12]:
tmodel.fit(X=X_train.values, treatment=treatment_train, y=y_train)

/Users/ogrobertino/new_diploma/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 370, number of negative: 1010
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000535 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 1380, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.268116 -> initscore=-1.004203
[LightGBM] [Info] Start training from score -1.004203


/Users/ogrobertino/new_diploma/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 909, number of negative: 1211
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 2120, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.428774 -> initscore=-0.286857
[LightGBM] [Info] Start training from score -0.286857
